In [1]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64
import os

In [2]:
import gdown

url = "https://drive.google.com/uc?id=1A0N8LXDorJVSPVdc6oLDNy2OMyGcPyi2"
output = "kecemasan.csv"
gdown.download(url, output, quiet=False)
import pandas as pd

stress = pd.read_csv("kecemasan.csv")
stress.info()

Downloading...
From: https://drive.google.com/uc?id=1A0N8LXDorJVSPVdc6oLDNy2OMyGcPyi2
To: /content/kecemasan.csv
100%|██████████| 1.29M/1.29M [00:00<00:00, 72.8MB/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2838 non-null   object
 1   label   2838 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 44.5+ KB


In [3]:
import pandas as pd

data = pd.read_csv("kecemasan.csv") # Assign the dataframe to 'data'
data

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1
...,...,...
2833,"* Her, a week ago: Precious, how are you? (I i...",0
2834,I don't have the ability to cope with it anymo...,1
2835,In case this is the first time you're reading ...,0
2836,Do you find this normal? They have a good rela...,0


In [4]:
data['text']

,text
0,"He said he had not felt that way before, sugge..."
1,"Hey there r/assistance, Not sure if this is th..."
2,My mom then hit me with the newspaper and it s...
3,"until i met my new boyfriend, he is amazing, h..."
4,October is Domestic Violence Awareness Month a...
...,...
2833,"* Her, a week ago: Precious, how are you? (I i..."
2834,I don't have the ability to cope with it anymo...
2835,In case this is the first time you're reading ...
2836,Do you find this normal? They have a good rela...


In [5]:
text_test = data['text'][1]

text_test

"Hey there r/assistance, Not sure if this is the right place to post this.. but here goes =) I'm currently a student intern at Sandia National Labs and working on a survey to help improve our marketing outreach efforts at the many schools we recruit at around the country. We're looking for current undergrad/grad STEM students so if you're a STEM student or know STEM students, I would greatly appreciate if you can help take or pass along this short survey. As a thank you, everyone who helps take the survey will be entered in to a drawing for chance to win one of three $50 Amazon gcs."

Mengambil data dari text ke 4 dan menginisialisasikan ke dalam text_test untuk proses test

In [ ]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(text_test)

endpoint = "http://localhost:8080/v1/models/stress-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Stress" if prediction_value > 0.6 else "Tidak Stress"
else:
    result = "Error: No predictions found in response."

print(result)

Tidak Stress


prepare_json() untuk mempersiapkan data input dalam json string yang akan digunakan untuk prediksi

Melakukan prediksi dari text_test yang dideploy pada api endpoint, apabila prediksi lebih besar dari 0.6 maka akan menghasilkan prediksi dari text "Stress", apabila dibawah 0.6 maka tidak stress